In [1]:
!rm -rf data
!rm -rf MixcraftDataset
!rm -rf testingdatapaper/
!rm -rf testingFiles/
!rm -rf dataFiles/

!git clone https://github.com/SutirthaChakraborty/MatchMiDi.git



!pip install py_midicsv
!pip install python-Levenshtein
!unzip /content/MatchMiDi/data.zip
!unzip /content/MatchMiDi/MixcraftDataset.zip
!pip install progressbar
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')




Cloning into 'MatchMiDi'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=148898 sha256=301d6852be30910d69bb99652ac9c74023a340eaeeb688712b540e979bb45c38
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
Archive:  /content/MatchMiDi/data.zip
  inflating: data/Sco_01_Jupiter_vn_vc.mid  
  inflating: data/Sco_02_Sonata_vn_vn.mid  
  inflating: data/Sco_03_Dance_fl_cl.mid  
  inflating: data/Sco_04_Allegro_fl_fl.mid  
  inflating: data/Sco_05_Entertainer_tpt_tpt.mid  
  inflating: data/Sco_06_Entertainer_sax_sax.mid  
  inflating: data/Sco_07_GString_t

In [2]:
import Levenshtein
def distance(tar,total):
    k=[]
    for i in tqdm(range(len(total)-len(tar))):
        k.append(Levenshtein.distance(str(tar),str(total[i:i+len(tar)])))
    # plt.plot(k)
    block=k.index(min(k))
    print("Minimum value= ",min(k))
    # print("time =", total['Timestamp'].iloc[block+1])
    return k,block


In [3]:
import py_midicsv as pm
import pandas as pd
import numpy as np
import glob
total=pd.DataFrame()

dataSet=pd.DataFrame()



t=1
files=[]
timediff=[]
pitchData=[]


for file in glob.glob("/content/data/*.mid"):
    files.append(file)
files=sorted(files)


for file in tqdm(files):
    csv_string1 = pm.midi_to_csv(file)
    content = [x.strip().split(', ') for x in csv_string1] 
    data=pd.DataFrame(content)
    data.columns=['A','Timestamp','NotePress','Track','Pitch','Velocity','G']
    y=pd.DataFrame([t] * data.shape[0])
    y.columns=['FileID']
    data=pd.concat([data,y],axis=1)
    total=total.append(data, ignore_index=True)
    t+=1
    total=total[15:-2]
    a=list(total.Track.unique())
    a=filter(None, a) # Remove nulls
    target_Tracks = [ i for i in a if i.isnumeric() ]    # Calculates the number of tracks
    for i in target_Tracks:


        tar=total.loc[total['Track'] == i]
        tar=tar.loc[(tar['NotePress'] == 'Note_off_c') | (tar['NotePress'] == 'Note_on_c') ]
    
        dataSet=dataSet.append(tar, ignore_index=True)
        if(int(i)>-1 and int(i)<=5):
            name= str(i)+".csv"
            tar.to_csv(name)

        if(len(list(np.diff(np.array(tar['Timestamp']).astype(float))))>0):
            pidata=list(tar['Pitch'].astype(float))
            pitchData=pitchData+pidata
            xx=[0]+list(np.diff(np.array(tar['Timestamp']).astype(float)))
            b = (xx - np.min(xx))/np.ptp(xx)
            timediff=timediff+list(b)


len(timediff),len(pitchData)

d=pd.DataFrame(timediff)
d.columns=['TimeDiff']
d.to_csv('timediff.csv')

d=pd.DataFrame(pitchData)
d.columns=['pitchData']
d.to_csv('pitchData.csv')


 


In [16]:


target_path='/content/MixcraftDataset/File - 24 - 2 sec.mid'

import py_midicsv as pm
import pandas as pd
import numpy as np
import glob

tar_timediff=[]
tar_pitchData=[]


timestamp_collector=[]
file_collector=[]
track_collector=[]


csv_string1 = pm.midi_to_csv(target_path)
content = [x.strip().split(', ') for x in csv_string1] 
data=pd.DataFrame(content)

data.columns=['A','Timestamp','NotePress','Track','Pitch','Velocity','G']
total=data

t+=1
total=total[15:-2]
a=list(total.Track.unique())
a=filter(None, a) # Remove nulls
target_Tracks = [ i for i in a if i.isnumeric() ]    # Calculates the number of tracks
for i in tqdm(target_Tracks):

    tar=total.loc[total['Track'] == i]
    tar=tar.loc[(tar['NotePress'] == 'Note_off_c') | (tar['NotePress'] == 'Note_on_c') ]
  
    if(len(list(np.diff(np.array(tar['Timestamp']).astype(float))))>0):
        pidata=list(tar['Pitch'].astype(float))
        tar_pitchData=tar_pitchData+pidata

        xx=[0]+list(np.diff(np.array(tar['Timestamp']).astype(float)))
        b = (xx - np.min(xx))/np.ptp(xx)
        tar_timediff=tar_timediff+list(b)

        filename = str(i)+"time.csv"
        d=pd.DataFrame(tar_timediff)
        d.columns=['tar_timediff']
        d.to_csv(filename)

        filename = str(i)+"pitch.csv"
        d=pd.DataFrame(tar_pitchData)
        d.columns=['pitchData']
        d.to_csv(filename)

        targetfile=str(i)+".csv"
        datasetfile=pd.read_csv(targetfile)

        k,block=distance(pidata,list(datasetfile.Pitch))
        # print(block)
        # b = (k - np.min(k))/np.ptp(k)
        # plt.plot(b)
        # plt.show()
        file_collector.append(datasetfile['FileID'].iloc[block+1])
        # print("Timestamp = ",total['Timestamp'].iloc[block+1])
        timestamp_collector.append(datasetfile['Timestamp'].iloc[block+1])
        # print("Track = ",total['Track'].iloc[block+1])
        track_collector.append(datasetfile['Track'].iloc[block+1])



# len(tar_timediff),len(tar_pitchData)
track_collector,timestamp_collector,file_collector


Minimum value=  20



Minimum value=  8



Minimum value=  16



Minimum value=  16



([0, 1, 2, 3], [30720, 70080, 29760, 29760], [24, 3, 24, 24])